# Behavioral Cloning

---

**Behavioral Cloning Project**

The goals / steps of this project are the following:

- Use the simulator to collect data of good driving behavior
- Build, a convolution neural network in Keras that predicts steering angles from images
- Train and validate the model with a training and validation set
- Test that the model successfully drives around track one without leaving the road
- Summarize the results with a written report

---

## Required Files

### 1. Are all required files submitted?

- `model.py` Code to training the model.
- `drive.py` Code to driving the car.
- `model.h5` Trained network data.
- `writeup_report` This notebook.
- `video.mp4` A video recording of my car.

## Quality of Code

### 1. Is the code functional?

You can run the code by run

```sh
python drive.py model.h5
```

### 2. Is the code usable and readable?

My code has the generator function named `generator`, it store the csv data in memory and read the image data when training the model.  
It also contains comments to explain each step and some functions.

## Model Architecture and Training Strategy

### 1. Has an appropriate model architecture been employed for the task?

My model is based on VGGNet, it has the convolution layers and RELU layers,  
and I've added 3 layers at the first of network:

```python
    model.add(Cropping2D(cropping=((64, 24), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: (x / 127.5) - 1, input_shape=(72, 320, 3)))
    model.add(AveragePooling2D(pool_size=(2, 4)))
```

They normalize the data in the model.

### 2. Has an attempt been made to reduce overfitting of the model?

Based on VGGNet, after every pooling layer and activation layer, there are a dropout layer to reduce overfitting.

### 3. Have the model parameters been tuned appropriately?

Admin optimizer used, it's really easy to use.

### 4. Is the training data chosen appropriately?

I've drived for:

- Two laps in the center of road.
- One lap of recovery driving from the sides.
- If the trained car drive out of road, get more samples at this position. 

## Architecture and Training Documentation

### 1. Is the solution design documented?

My model is based on VGGNet because I think in this case, the data is different and the data set is large. And the VGGNet can detect all the features in the images.  

First, I've added three layers in the top of network,

- Cropping2D: Reduce the image size and keep the helpful data.
- Lambda: Normalize data.
- AveragePooling2D: Further reduce the image size, like `cv2.resize` but it's in the network.

![Cropping Image](notebook_images/cropping1.png)
![Cropping Image](notebook_images/cropping2.png)
![Cropping Image](notebook_images/cropping3.png)

And then, I've changed the bottom of full connection layers to adptive this project: 43 classes.  
But the result isn't very well because the predict result is linear, but the classes is discrete.  
So I've changed the last layer to a tanh, it will generate the result in range (-1, 1), lucky, most the steering angle are in this range. So I cut the steering angle in samples (in `def process_angle(angle, speed)` function), and used the result of tanh activation directly, use `mse` for the loss function.

When I training the model ,I found that the model like predict the angle near the `camera_correction` parameter, even when turning rather than correcting position, certainly, it will get the less loss.  
I think it's caused by the count of zero steering samples, so I've droped some zero samples:

```python
# Drop half of 0 steering samples
if float(row[6]) != 0 and random.random() < 0.5:
    samples.append(row)
```

And then, I've added more samples at the bad driving position, add convolution layer or tune the features in every convlotion layer when I think the model forgot some knowledge.

At last, I get a good model that the car tune to the correct direction in everywhere of the road.  
But my car still can't drive safely over one lap, I found that because the understeer.  
Then, I have try to modified the `drive.py` to change the drive strategy.


Use 50% of the last steering angle to smooth the driving, and counter the understeer, because it will enlarge the steering angle when continuous turning.  
```python
# smooth steering
steering_angle = self.last_steering_angle * 0.5 + steering_angle
self.last_steering_angle = steering_angle
```

If the car wants steering a big angle, brake like a human. (I've increase the desired speed because it's too slow for testing)  
*I've tried to predict the steering angle, threshold speed (brake) in model at the same time. But it's hardly to balance their weights for the loss function.*  
```python
if abs(steering_angle) > 0.05:
    steering_angle = steering_angle * 2
    controller.set_desired(set_speed / 2)
else:
    controller.set_desired(set_speed)
```

I have change the training code to train the model epoch by epoch, the validation loss always increase at the third of fourth time.  
```python
if os.path.exists(FLAGS.model):
    model = load_model(FLAGS.model)
else:
    model = CNN()
```

At last, my car can run over one lap.


### 2. Is the model architecture documented?

My final model is like this:  
*You can also see `def CNN()` in the code.*

```
    +--------------+
    |   Cropping   |
    +--------------+
           |
+----------------------+
| Normalization Lambda |
+----------------------+
           |
+----------------------+
|    Average Pooling   |
|      2x4 kernel      |
+----------------------+
           |
+----------------------+
|     Convolution      |
|    36@ 5x5 kernel    |
+----------------------+
           |
+----------------------+
|     Convolution      |
|    48@ 5x5 kernel    |
+----------------------+
           |
+----------------------+
|     Max Pooling      |
|      2x2 kernel      |
+----------------------+
           |
     +----------+
     |   RELU   |
     +----------+
           |
    +-------------+
    |   Dropout   |
    +-------------+
           |
+----------------------+
|     Convolution      |
|    64@ 3x3 kernel    |
+----------------------+
           |
+----------------------+
|     Convolution      |
|    64@ 3x3 kernel    |
+----------------------+
           |
+----------------------+
|     Max Pooling      |
|      2x2 kernel      |
+----------------------+
           |
     +----------+
     |   RELU   |
     +----------+
           |
    +-------------+
    |   Dropout   |
    +-------------+
           |
    +-------------+
    |   Flatten   |
    +-------------+
           |
 +---------------------+
 |   Fully connected   |
 |    100 with RELU    |
 +---------------------+
           |
    +-------------+
    |   Dropout   |
    +-------------+
           |
 +---------------------+
 |   Fully connected   |
 |     50 with RELU    |
 +---------------------+
           |
    +-------------+
    |   Dropout   |
    +-------------+
           |
 +---------------------+
 |   Fully connected   |
 |     10 with RELU    |
 +---------------------+
           |
    +-------------+
    |   Dropout   |
    +-------------+
           |
 +---------------------+
 |   Fully connected   |
 |     1 with Tanh     |
 +---------------------+
           |
    +--------------+
    |   Output 1   |
    +--------------+
```



### 3. Is the creation of the training dataset and training process documented?

The training process is just in the solution design and training data chosen section.  
Here is the hard positions where I've run multiple times:

![Hard Position](notebook_images/hard1.png)
![Hard Position](notebook_images/hard2.png)
![Hard Position](notebook_images/hard3.png)


## Simulation

### 1. Is the car able to navigate correctly on test data?

Please look at the `video.mp4`, or run the drive code with simulator on 640x480, Fastest mode.

---

## Code

### model.py

In [ ]:
import csv
import os
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from scipy.misc import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D, AveragePooling2D

%matplotlib inline

# command line flags
try:
    flags
except Exception:
    flags = tf.app.flags
    FLAGS = flags.FLAGS

    flags.DEFINE_string('data_dir', 'dataset', 'Directory of the dataset and driving log.')
    flags.DEFINE_string('camera_correction', 0.15, 'Steering correction for the side camera images.')
    flags.DEFINE_string('batch_size', 64, 'Batch_size of training.')
    flags.DEFINE_string('epoch', 3, 'Epoch of training.')
    flags.DEFINE_string('model', 'model.h5', 'Saved model file.')


with open(os.path.join(FLAGS.data_dir, 'driving_log.csv'), 'r') as f:
    reader = csv.reader(f)
    samples = []
    for row in reader:
        # Drop half of 0 steering samples
        if float(row[6]) != 0 and random.random() < 0.5:
            samples.append(row)

# Split data set to train and validation set
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


def generator(samples, batch_size=FLAGS.batch_size):
    def process_angle(angle, speed):
        """
        All the angles are between -1.2 to 1.2, cut them to [-1, 1] one hot data.
        And I have also increase the angle when I have speed down to pass the curve.
        """
            
        if angle > 1.0:
            return 1.0
        elif angle < -1.0:
            return -1.0
        else:
            return angle

    def fix_path(path):
        """ Because I've recoed the data in Windows, so need fix the image path in the log file. """
        path = path.strip()
        path = path[path.find('IMG'):]
        path = os.path.join(*path.split('\\'))
        return path

    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset + batch_size]

            images = []
            actions = []
            for batch_sample in batch_samples:
                speed = float(row[6])
                steering_center_angle = float(row[3])

                img_center = imread(os.path.join(FLAGS.data_dir, fix_path(row[0])))
                img_left = imread(os.path.join(FLAGS.data_dir, fix_path(row[1])))
                img_right = imread(os.path.join(FLAGS.data_dir, fix_path(row[2])))
                
                # Add images, fliped images and actions to data set
                images.extend([img_center, img_left, img_right])
                images.extend([np.fliplr(img_center), np.fliplr(img_left), np.fliplr(img_right)])
                actions.extend([
                    process_angle(steering_center_angle, speed),
                    process_angle(steering_center_angle + FLAGS.camera_correction, speed),
                    process_angle(steering_center_angle - FLAGS.camera_correction, speed)
                ])
                actions.extend([
                    process_angle(-steering_center_angle, speed),
                    process_angle(-steering_center_angle - FLAGS.camera_correction, speed),
                    process_angle(-steering_center_angle + FLAGS.camera_correction, speed)
                ])

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(actions).reshape(-1, 1)
            yield shuffle(X_train, y_train)

                                        
def CNN():
    """ My CNN based on VGGNet. """
    model = Sequential()
    model.add(Cropping2D(cropping=((64, 24), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: (x / 127.5) - 1, input_shape=(72, 320, 3)))
    # input: 72x320 images with 3 channels
    # Using one average pooling to change the image size smaller
    model.add(AveragePooling2D(pool_size=(2, 4)))

    # this applies 32 convolution filters of size 3x3 each.
    model.add(Convolution2D(36, 5, 5, border_mode='valid', input_shape=(72, 320, 3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(48, 5, 5))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('tanh'))
    
    model.compile(loss='mse', optimizer='adam')
    return model


# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=FLAGS.batch_size)
validation_generator = generator(validation_samples, batch_size=FLAGS.batch_size)

if os.path.exists(FLAGS.model):
    model = load_model(FLAGS.model)
else:
    model = CNN()
    
# Because I've flip all the image and 3 directions, so the size of data set must be multiple by 6
history_object = model.fit_generator(
    train_generator,
    samples_per_epoch=len(train_samples) * 6,
    validation_data=validation_generator,
    nb_val_samples=len(validation_samples) * 6,
    nb_epoch=FLAGS.epoch,
    verbose=1
)

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

# Save the model
model.save(FLAGS.model)

### drive.py

In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.
        self.last_steering_angle = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement, steering_angle):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error
        
        # smooth steering
        steering_angle = self.last_steering_angle * 0.5 + steering_angle
        self.last_steering_angle = steering_angle

        return self.Kp * self.error + self.Ki * self.integral, steering_angle


controller = SimplePIController(0.05, 0.002)
set_speed = 15
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        result = model.predict(image_array[None, :, :, :], batch_size=1)
        predict_steering_angle = result[0][0]

        throttle, steering_angle = controller.update(float(speed), predict_steering_angle)
        if abs(steering_angle) > 0.05:
            steering_angle = steering_angle * 2
            controller.set_desired(set_speed / 2)
        else:
            controller.set_desired(set_speed)

        print(predict_steering_angle, steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Remote Driving')
    parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )
    parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )
    args = parser.parse_args()

    # check that model Keras version is same as local Keras version
    f = h5py.File(args.model, mode='r')
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    model = load_model(args.model)

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)
